<a href="https://colab.research.google.com/github/karthikpaidi359/fake-news-detector/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files

# This will open a file upload dialog
uploaded = files.upload()


Saving True.csv to True.csv
Saving Fake.csv to Fake.csv


In [2]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# 🔹 Step 1: Load the CSVs
df_fake = pd.read_csv("Fake.csv")
df_true = pd.read_csv("True.csv")

# 🔹 Step 2: Label the data
df_fake["label"] = 0  # Fake news = 0
df_true["label"] = 1  # Real news = 1

# 🔹 Step 3: Combine and clean
df = pd.concat([df_fake, df_true])
df = df[["text", "label"]].dropna()  # Use only 'text' and 'label'

# 🔹 Step 4: Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    df["text"], df["label"], test_size=0.2, random_state=42
)

# 🔹 Step 5: Vectorize text using TF-IDF
tfidf = TfidfVectorizer(stop_words="english", max_df=0.7)
X_train_tfidf = tfidf.fit_transform(X_train)

# 🔹 Step 6: Train a Logistic Regression model
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# 🔹 Step 7: Save model and vectorizer as .pkl files
with open("fake_news_model.pkl", "wb") as f:
    pickle.dump(model, f)

with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)

print("✅ Model trained and saved successfully!")


✅ Model trained and saved successfully!


In [3]:
from google.colab import files

# Download the trained model
files.download("fake_news_model.pkl")

# Download the TF-IDF vectorizer
files.download("tfidf_vectorizer.pkl")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [4]:
!pip install gradio


In [5]:
import gradio as gr
import pickle

# Load trained model
with open("fake_news_model.pkl", "rb") as f:
    model = pickle.load(f)

with open("tfidf_vectorizer.pkl", "rb") as f:
    vectorizer = pickle.load(f)


In [6]:
def predict_news(text):
    if not text.strip():
        return "⚠️ Please enter some news content."
    try:
        vect_text = vectorizer.transform([text])
        pred = model.predict(vect_text)[0]
        return "✅ REAL News" if pred == 1 else "❌ FAKE News"
    except Exception as e:
        return f"🚫 Error: {e}"


In [7]:
interface = gr.Interface(
    fn=predict_news,
    inputs=gr.Textbox(lines=10, placeholder="Paste news content here..."),
    outputs="text",
    title="📰 Fake News Detector",
    description="Paste a news article or headline to check if it's real or fake."
)

interface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c7026368516c27fba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [8]:
def predict_news(text):
    if not text.strip():
        return "⚠️ Please enter some news content."
    vect_text = vectorizer.transform([text])
    pred = model.predict(vect_text)[0]
    return "✅ REAL News" if pred == 1 else "❌ FAKE News"


In [9]:
def predict_news(text):
    if not text.strip():
        return "⚠️ Please enter some news content."
    vect_text = vectorizer.transform([text])
    prob = model.predict_proba(vect_text)[0]
    label = model.predict(vect_text)[0]
    confidence = max(prob)
    result = "✅ REAL News" if label == 1 else "❌ FAKE News"
    return f"{result} (Confidence: {confidence:.2f})"


In [10]:
interface.launch()


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3c7026368516c27fba.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
